Common settings

In [1]:
# import of auto-sklearn version that supports SMOTE, source has to be downloaded localy
if True:
    import sys
    sys.path.insert(0, "../my_autosklearn")


SEED = 0
K_FOLDS = 5
BASE_LEARNER = "RF"

XGB settings

In [2]:
from math import floor, sqrt
from xgboost import XGBClassifier, XGBRFRegressor


# TODO: figure out parameters, especially objective 
def get_XGBModel(X, y, base_learner, task):
    m = len(X[0])

    if task == "classification":
        XGBModel = XGBClassifier
        objective = "binary:logistic"
    elif task == "regression":
        XGBModel = XGBRFRegressor
        objective = "binary:logistic"
    else:
        assert False, f"Wrong arguent: {task}."

    if base_learner == "RF":
        return XGBModel(
            learning_rate = 0.2, # 0.0 - 1.0, log, not sure if each forest can have its own, it might lead to overfitting anyway
            max_depth = 20, # 2 - 50, 6 - 20 is another more conservative option
            subsample = 0.63, # 0.0 - 1.0
            # u can set only one of the colsample_by*
            # colsample_bytree: Optional[float] = None,
            # colsample_bylevel: Optional[float] = None,
            colsample_bynode = floor(sqrt(m))/m, # m..num_of_features, 0 - m, log?
            n_estimators = 10, # 100 - 500 = number of random forests in booster
            num_parallel_tree = 10, # 100 - 500 = number of trees in each random forest
            reg_lambda = 0, # -10. - 10.0, log = prunning of trees, higher value -> more prunning, not sure if negative values do anything
            min_child_weight = 2, # 0.0 - 10.0, log, higher value -> less options to choose from when selecting new nodes in trees
            objective = objective, # list at https://xgboost.readthedocs.io/en/stable/parameter.html, search for objective
            seed=SEED, 
            seed_per_iteration=True,
        )
    elif base_learner == "DecisionTree":
        return XGBModel(
            objective=objective,
            seed=SEED,
            seed_per_iteration=True
        )
    else:
        assert False, f"Wrong arguent: {base_learner}."

ImportError: cannot import name 'XGBRegression' from 'xgboost' (/home/karilli/PV056/project/venv/lib/python3.10/site-packages/xgboost/__init__.py)

Auto-sklearn settings

In [ ]:

from autosklearn.pipeline.components.base import AutoSklearnPreprocessingAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, UNSIGNED_DATA, INPUT
from ConfigSpace.configuration_space import ConfigurationSpace


class NoPreprocessing(AutoSklearnPreprocessingAlgorithm):
    def __init__(self, **kwargs):
        for key, val in kwargs.items():
            setattr(self, key, val)

    def fit(self, X, Y=None):
        return self

    def transform(self, X):
        return X

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            "shortname": "NoPreprocessing",
            "name": "NoPreprocessing",
            "handles_regression": True,
            "handles_classification": True,
            "handles_multiclass": True,
            "handles_multilabel": True,
            "handles_multioutput": True,
            "is_deterministic": True,
            "input": (SPARSE, DENSE, UNSIGNED_DATA),
            "output": (INPUT,),
        }

    @staticmethod
    def get_hyperparameter_search_space(feat_type=None, dataset_properties=None):
        return ConfigurationSpace()


import autosklearn.pipeline.components.data_preprocessing
autosklearn.pipeline.components.data_preprocessing.add_preprocessor(NoPreprocessing)

In [ ]:
from autosklearn.pipeline.components.base import AutoSklearnClassificationAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, UNSIGNED_DATA, INPUT
from ConfigSpace.configuration_space import ConfigurationSpace


class XGBClassifier_(AutoSklearnClassificationAlgorithm):
    def __init__(self, **kwargs):
        self.estimator = None
        for key, val in kwargs.items():
            setattr(self, key, val)

    def fit(self, X, y, sample_weight=None):
        self.estimator = get_XGBModel(X, y, BASE_LEARNER, "classification").fit(X, y, sample_weight=sample_weight)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    def predict_proba(self, X):
        if self.estimator is None:
            raise NotImplementedError()
        return self.estimator.predict_proba(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            "shortname": "xgboost",
            "name": "xgboost",
            "handles_regression": False,
            "handles_classification": True,
            "handles_multiclass": True,
            "handles_multilabel": True,
            "handles_multioutput": True,
            "is_deterministic": True,
            "input": (SPARSE, DENSE, UNSIGNED_DATA),
            "output": (INPUT,),
        }

    @staticmethod
    def get_hyperparameter_search_space(feat_type=None, dataset_properties=None):
        return ConfigurationSpace()

import autosklearn.pipeline.components.classification
autosklearn.pipeline.components.classification.add_classifier(XGBClassifier_)

In [ ]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import roc_auc, recall_weighted


from sklearn.model_selection import StratifiedKFold
from os.path import exists
import shutil


def precision_minority():
    pass  # TODO


TEMP = "temp_folder"

feature_preprocessor = [
    'densifier', 'extra_trees_preproc_for_classification', 'fast_ica', 'feature_agglomeration', 
    'kernel_pca', 'kitchen_sinks', 'liblinear_svc_preprocessor', 'no_preprocessing', 'nystroem_sampler', 'pca', 
    'polynomial', 'random_trees_embedding', 'select_percentile_classification', 'select_rates_classification', 
    'truncatedSVD'
]


# TODO: figure out parameters
def get_AutoSklearnClassifier(X, y, task):
    if task == "imbalance data":
        metric=[roc_auc, precision_minority],
        include={
            "data_preprocessor": ["no_preprocessing"],
            "balancing": ["none", "weighting", "SVMSMOTE"],
            "feature_preprocessor": ["no_preprocessing"],
            "classifier": ["xgboost"]
        },
    elif task == "inadequate features":
        metric=[roc_auc, recall_weighted],
        include={
            "data_preprocessor": ["no_preprocessing"],
            "balancing": ["none"],
            "feature_preprocessor": feature_preprocessor,
            "classifier": ["xgboost"]
        },
    else:
        assert False, f"Wrong arguent: {task}."

    if exists(TEMP):
        shutil.rmtree(TEMP, ignore_errors=True)
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)
    return AutoSklearnClassifier(
        time_left_for_this_task=15*60,
        metric=metric,
        initial_configurations_via_metalearning=0,
        ensemble_kwargs={"ensemble_size": 1},
        include=include,
        resampling_strategy=skf,
        seed=SEED,
        tmp_folder=TEMP,
        delete_tmp_folder_after_terminate=True,
        n_jobs=8,
        memory_limit=6000,
        per_run_time_limit=3*60
    ).fit(X, y)

Dataset loading

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np


# TODO: find all of the missing datasets
IDS = [
    # ("breast-w", 15),
    # ("credit-approval", 29),
    # ("credit-g", 31),
    # ("diabetes", 37),
    # ("sick", 38),
    # ("spambase", 44),
    # ("tic-tac-toe", 50),
    # ("electricity", 151),
    # ("vowel", 307),
    # ("pc4", 1049),
    # ("pc3", 1050),
    ("JM1", 1053),
    # ("KC2", 1063),
    # ("kc1", 1067),
    # ("pc1", 1068),
    # ("bank-marketing", 1461),
    # ("blood-transfusion-service-center", 1464),
    # ("ilpd", 1480),
    # ("madelon", 1485),
    # ("nomao", 1486),
    # ("ozone-level-8hr", 1487),
    # ("phoneme", 1489),
    # ("qsar-biodeg", 1494),
    # ("adult", 1590),
    # ("Bioresponse", 4134),
    # ("cylinder-bands", 6332),
    # ("dresses-sales", 23381),
    # ("numerai28.6", 23517),
    # ("churn", 40701),
    # ("wilt", 40983),
    # ("climate-model-simulation-crashes", 40994),
]


DATASETS = []
for dataset_name, dataset_id in IDS:
    data = fetch_openml(data_id=dataset_id, parser="auto", as_frame=True)

    if data.frame.shape[0] < 1000:
        continue
    if len(data.target_names) != 1:
        continue
    target = data.target_names[0]
    if len(data.frame[target].unique()) != 2:
        continue

    X = data.frame.drop(columns=[target])
    (l1, l2), (c1, c2) = np.unique(data.frame[target], return_counts=True)
    (c1, l1), (c2, l2) = sorted(((c1, l1), (c2, l2)))
    y = data.frame[target] == l1
    DATASETS.append((dataset_name, X, y))

[name for name, _, _ in DATASETS]

Utils for validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    recall_score, 
    precision_score, 
    f1_score, 
    fbeta_score, 
    roc_auc_score,
    roc_curve
)


# TODO: which metric and how to compute for which task?
# TODO: measure train time and inference time for each fold
def validate(X, y, model):
    res = {}
    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)
    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if hasattr(model, "refit"):
            model = model.refit(X_train, y_train)
        else:
            # XGB docs says that calling fit() multiple times
            # will cause re-fit, unless xgb_model parameter
            # is provided explicitly
            model = model.fit(X_train, y_train)

        y_prob = model.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        y_pred = thresholds[(tpr - fpr).argmax()] < y_prob
        res[fold] = {
            "auc_roc": roc_auc_score(y_test, y_prob),
            "weighted_precision": precision_score(y_test, y_pred, average='weighted', zero_division=0.0),
            "macro_precision": precision_score(y_test, y_pred, average='macro', zero_division=0.0),
            "weighted_recall": recall_score(y_test, y_pred, average='weighted', zero_division=0.0),
            "macro_recall": recall_score(y_test, y_pred, average='macro', zero_division=0.0),
            "weighted_f1": f1_score(y_test, y_pred, average='weighted', zero_division=0.0),
            "macro_f1": f1_score(y_test, y_pred, average='macro', zero_division=0.0),
            "weighted_f2": fbeta_score(y_test, y_pred, beta=2, average='weighted', zero_division=0.0),
            "macro_f2": fbeta_score(y_test, y_pred, beta=2, average='macro', zero_division=0.0),
            "y_pred": [float(y) for y in y_pred],
            "y_prob": [float(y) for y in y_prob],
            "y_test": [float(y) for y in y_test],
        }
    return res

Data imbalance

In [ ]:
from imblearn.datasets import make_imbalance


IMBALANCED_DATSETS = []
ratios = [0.5, 0.25] + [r/100 for r in range(1, 21)]
for name, X, y in DATASETS:
    for ratio in sorted(ratios, reverse=True):
        (l1, l2), (c1, c2) = np.unique(data.frame[target], return_counts=True)
        (c1, l1), (c2, l2) = sorted(((c1, l1), (c2, l2)))
        new_minority_count = int(c2 * ratio)
        if c1 < new_minority_count:
            continue
        X, y = make_imbalance(X, y, sampling_strategy={0: c2, 1: new_minority_count}, random_state=SEED)
        IMBALANCED_DATSETS.append((name, ratio, X, y))

[(name, ratio) for name, ratio, _, _ in IMBALANCED_DATSETS]

In [ ]:
imbalance_scores = {}
for name, ratio, X, y in IMBALANCED_DATSETS:
    model = get_AutoSklearnClassifier(X, y, "imbalanced data")
    imbalance_scores[f"{name}(ratio={ratio:.2f})"] = validate(X, y, model)

imbalance_scores

Noisy data

In [ ]:
NOISY_DATSETS = []
noise_amount = [a/100 for a in range(1, 11)]
for name, X, y in DATASETS:
    for noise in sorted(noise_amount):
        # TODO: add 1% of noise to X each iteration, dont over write previous noise
        # X = ...
        pass

[(name, noise) for name, noise, _, _ in NOISY_DATSETS]

In [ ]:
class NoisyXGBClassifier:
    def fit(self, X, y):
        X_train, y_train = self.remove_noise(X, y)
        model = get_XGBModel(X_train, y_train, BASE_LEARNER, "classification")
        model = model.fit(X_train, y_train)
        return model

    def remove_noise(self, X, y):
        model = get_XGBModel(X, y, BASE_LEARNER, "regression")
        model = model.fit(X, y)
        # TODO: create a new dataset instead of renaming labels
        X_transformed, y_transformed = X, model.predict(X)
        return X_transformed, y_transformed


noisy_scores = {}
for name, noise, X, y in NOISY_DATSETS:
    model = NoisyXGBClassifier()
    noisy_scores[f"{name}(noise={noise})"] = validate(X, y, model)

noisy_scores

semi-supervised

In [ ]:
HIDDEN_DATSETS = []
hidden_amount = [a/100 for a in range(1, 11)]
for name, X, y in DATASETS:
    for hidden in sorted(hidden_amount):
        # TODO: create hidden instances
        X, X_hidden = X, X
        HIDDEN_DATSETS.append((name, hidden, X, X_hidden, y))

[(name, hidden) for name, hidden, _, _, _ in HIDDEN_DATSETS]

In [ ]:
from sklearn.ensemble import BaggingClassifier


# TODO: how to set up bagging
N_ESTIMATORS_BAGGING = None
N_JOBS_BAGGING = None

for name, hidden, X, X_hidden, y in HIDDEN_DATSETS:
    while True: # TODO: while there are any hidden instances
        model = BaggingClassifier(
            estimator=get_XGBModel(X, y, BASE_LEARNER, "classification"),
            n_estimators = N_ESTIMATORS_BAGGING,
            n_jobs = N_JOBS_BAGGING,
            random_state = SEED
        )
        model.fit(X, y)
        y_hidden = model.predict_proba(X_hidden)
        # TODO: select instances from X_hidden that most of the ensemble agrees on,
        # add them together with labels to X, y and remove from X_hidden
    model = get_XGBModel(X, y, BASE_LEARNER, "classification")
    noisy_scores[f"{name}(noise={noise})"] = validate(X, y, model)


Inadequate features

In [ ]:
inadequate_features_scores = {}
for name, X, y in DATASETS:
    model = get_AutoSklearnClassifier(X, y, task="inadequate features")
    inadequate_features_scores[name] = validate(X, y, model)

inadequate_features_scores